In [1]:
import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [3]:
domain = """

resource("Wood").
resource("Food").
resource("Planks").
resource("Ration").

time(0..8).

agent(bob).
agent(woody).

action(wait).
action(gather("Wood")).
action(gather("Food")).
action(process_to("Wood", "Plank")).
action(process_to("Food", "Ration")).
action(give_to_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).
action(accept_from_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).


"""
solve([domain]);

Answer 1: { action(wait) action(gather("Food")) action(gather("Wood")) action(process_to("Food","Ration")) action(process_to("Wood","Plank")) agent(bob) agent(woody) resource("Food") resource("Planks") resource("Ration") resource("Wood") time(0) time(1) time(2) time(3) time(4) time(5) time(6) time(7) time(8) }
SAT 1


<block>:18:90-110: info: atom does not occur in any rule head:
  component(Component)

<block>:19:94-114: info: atom does not occur in any rule head:
  component(Component)



In [4]:
domain_rules = """

startTime(ST) :- time(ST), not time(ST - 1).
endTime(ET) :- time(ET), not time(ET + 1).


% inertia
obs_at(agent_inventory(Agent, Resource), T+1) :-
  time(T), time(T+1),
  obs_at(agent_inventory(Agent, Resource), T),
  not occ_at(agent_does(Agent, process_to(Resource, _)), T).
  %not occ_at((Resource), T).
  
% process
obs_at(agent_inventory(Agent, Processed), T+1) :-
  time(T), time(T+1),
  occ_at(agent_does(Agent, process_to(Resource, Processed)), T),
  obs_at(agent_inventory(Agent, Resource), T).

  
% gather
obs_at(agent_inventory(Agent, Resource), T+1) :-
  time(T), time(T+1),
  occ_at(agent_does(Agent, gather(Resource)), T).

"""
solve([domain_rules]);

Answer 1: {  }
SAT 1


<block>:3:18-26: info: atom does not occur in any rule head:
  time(ST)

<block>:3:32-44: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:4:16-24: info: atom does not occur in any rule head:
  time(ET)

<block>:4:30-42: info: atom does not occur in any rule head:
  time((ET+1))

<block>:9:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:9:12-21: info: atom does not occur in any rule head:
  time((T+1))

<block>:11:7-60: info: atom does not occur in any rule head:
  occ_at(agent_does(#X0,process_to(#X1,#P2)),#X3)

<block>:16:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:16:12-21: info: atom does not occur in any rule head:
  time((T+1))

<block>:17:3-64: info: atom does not occur in any rule head:
  occ_at(agent_does(Agent,process_to(Resource,Processed)),T)

<block>:23:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:23:12-21: info: atom does not occur in any rule head:
  time((T+1))

<block>:24:3-49: in

In [5]:
deontic_domain = """

contract(ec).

component("Lieferung Essen").
component("Lieferung Holz").

componentType("Sache").

componentState("Vertragsschluss").
componentState("Gewährleistung").
componentState("Erfüllung").

%state("Unmöglichkeit").
%state("Schlechtleistung").
%state("Verzug").

contract_component(ec, "Lieferung Essen").
contract_component(ec, "Lieferung Holz").

component_componentType("Lieferung Essen", "Sache").
component_componentType("Lieferung Holz", "Sache").

deo_at(contract_component_deadline(ec, "Lieferung Essen", 4), 0).
deo_at(contract_component_deadline(ec, "Lieferung Holz", 4), 0).
deo_at(contract_component_state(ec, "Lieferung Essen", "Vertragsschluss"), 0).
deo_at(contract_component_state(ec, "Lieferung Holz", "Vertragsschluss"), 0).

"""
solve([deontic_domain]);

Answer 1: { component("Lieferung Essen") component("Lieferung Holz") componentState("Erfüllung") componentState("Gewährleistung") componentState("Vertragsschluss") componentType("Sache") contract(ec) component_componentType("Lieferung Essen","Sache") component_componentType("Lieferung Holz","Sache") contract_component(ec,"Lieferung Essen") contract_component(ec,"Lieferung Holz") deo_at(contract_component_deadline(ec,"Lieferung Essen",4),0) deo_at(contract_component_deadline(ec,"Lieferung Holz",4),0) deo_at(contract_component_state(ec,"Lieferung Essen","Vertragsschluss"),0) deo_at(contract_component_state(ec,"Lieferung Holz","Vertragsschluss"),0) }
SAT 1


In [6]:
deontic_rules = """

% Inertia
deo_at(contract_component_deadline(Contract, Component, Deadline), T+1) :-
  time(T+1), time(T), deo_at(contract_component_deadline(Contract, Component, Deadline), T).
  
deo_at(contract_component_state(Contract, Component, State), T+1) :-
  time(T), time(T+1),
  deo_at(contract_component_state(Contract, Component, State), T),
  not imp_at(contract_component_state(Contract, Component, State), T+1).

imp_at(contract_component_state(Contract, Component, State), T) :-
  componentState(State), State != AlternativeState,
  deo_at(contract_component_state(Contract, Component, AlternativeState), T).

rul_component_at(transaction_from_to(Thing, Giver, Accepter), Component, T) :-
  occ_at(agent_does(Giver, give_to_because(Thing, Accepter, Component)), T),
  occ_at(agent_does(Accepter, accept_from_because(Thing, Giver, Component)), T),
  deo_at(contract_component_state(Contract, Component, "Vertragsschluss"), T).

deo_at(contract_component_state(Contract, Component, "Gewährleistung"), T+1) :-
  time(T+1),
  deo_at(contract_component_state(Contract, Component, "Vertragsschluss"), T),
  rul_component_at(transaction_from_to(_Thing, _Giver, _Accepter), Component, T).

rul_component_at(end("Gewährleistung"), Component, T) :-
  time(T),
  deo_at(contract_component_state(Contract, Component, "Gewährleistung"), T),
  rul_component_at(transaction_from_to(_Thing, _Giver, _Accept), Component, T-2).

deo_at(contract_component_state(Contract, Component, "Erfüllung"), T+1) :-
  deo_at(contract_component_state(Contract, Component, "Gewährleistung"), T),
  rul_component_at(end("Gewährleistung"), Component, T).

"""
solve([deontic_rules]);

Answer 1: {  }
SAT 1


<block>:5:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:5:14-21: info: atom does not occur in any rule head:
  time(T)

<block>:8:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:8:12-21: info: atom does not occur in any rule head:
  time((T+1))

<block>:13:3-24: info: atom does not occur in any rule head:
  componentState(State)

<block>:17:3-76: info: atom does not occur in any rule head:
  occ_at(agent_does(Giver,give_to_because(Thing,Accepter,Component)),T)

<block>:18:3-80: info: atom does not occur in any rule head:
  occ_at(agent_does(Accepter,accept_from_because(Thing,Giver,Component)),T)

<block>:22:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:27:3-10: info: atom does not occur in any rule head:
  time(T)



In [7]:
instance = """

occ_at(agent_does(bob, gather("Food")), 1).
occ_at(agent_does(woody, gather("Wood")), 1).

occ_at(agent_does(bob, process_to("Food", "Ration")), 2).
occ_at(agent_does(woody, process_to("Wood", "Plank")), 2).

occ_at(agent_does(bob, give_to_because("Ration", woody, "Lieferung Essen")), 3).
occ_at(agent_does(woody, accept_from_because("Ration", bob, "Lieferung Essen")), 3).
occ_at(agent_does(woody, give_to_because("Plank", bob, "Lieferung Holz")), 4).
occ_at(agent_does(bob, accept_from_because("Plank", woody, "Lieferung Holz")), 4).

"""
solve([instance]);

Answer 1: { occ_at(agent_does(bob,gather("Food")),1) occ_at(agent_does(bob,process_to("Food","Ration")),2) occ_at(agent_does(bob,accept_from_because("Plank",woody,"Lieferung Holz")),4) occ_at(agent_does(bob,give_to_because("Ration",woody,"Lieferung Essen")),3) occ_at(agent_does(woody,gather("Wood")),1) occ_at(agent_does(woody,process_to("Wood","Plank")),2) occ_at(agent_does(woody,accept_from_because("Ration",bob,"Lieferung Essen")),3) occ_at(agent_does(woody,give_to_because("Plank",bob,"Lieferung Holz")),4) }
SAT 1


In [8]:
solve([
    domain,
    domain_rules,
    deontic_domain,
    deontic_rules,
    instance,
    "#show obs_at/2.",
    "#show occ_at/2.",
    "#show deo_at/2.",
    "#show rul_component_at/3.",
], sep='\n');

Answer 1: {
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),0)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),1)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),2)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),3)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),4)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),5)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),6)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),7)
deo_at(contract_component_deadline(ec,"Lieferung Essen",4),8)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),0)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),1)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),2)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),3)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),4)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),5)
deo_at(contract_component_deadline(ec,"Lieferung Holz",4),6)
deo